In [1]:
import base.epsilon_rho as epsilon_rho
import numpy as np
from base import metrics
import tensorflow as tf
%load_ext autoreload
%autoreload 2

2025-03-01 18:33:42.283810: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 18:33:42.284308: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 18:33:42.286778: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 18:33:42.294188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740821622.307047   19153 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740821622.31

In [2]:
from base import generator_gellmann
from base import generator_haar

def initExperiment(n):
    d = 2**n
    # Generate 6^n density matrices
    rho_list = generator_haar.generate_n_qubits_rho_haar(n)

    print(f"Generated {len(rho_list)} of {rho_list[0].shape} rho.")
    # ----------------------------
    single_qubits_projectors = generator_gellmann.generate_measurement_projector_gellmann(num_qubits = 1)
    measurement_operators = generator_gellmann.generate_measurement_operators_gellmann(single_qubits_projectors, n)
    print(f"Generated {len(measurement_operators)} of {measurement_operators[0].shape} M.")
    # ----------------------------
    # Generate epsilon
    epsilon = generator_haar.random_unitary(d)
    print(f"Generated {epsilon.shape} epsilon.")
    # Generate K list
    unitary = generator_haar.haar(d)
    kraus_operators = generator_haar.generate_kraus_operators(unitary)
    print(f"Generated {len(kraus_operators)} of {kraus_operators[0].shape} kraus operators.")
    return rho_list, measurement_operators, epsilon, kraus_operators


In [3]:
def calculate_rho2_lists(rho_list, epsilon, kraus_operators):
    rho2 = []
    rho2_kraus = []
    for rho in rho_list:
        rho2.append(epsilon_rho.calculate_from_unitary(rho, epsilon))
        rho2_kraus.append(epsilon_rho.calculate_from_kraus_operators(rho, kraus_operators))
    return rho2, rho2_kraus

In [4]:
import numpy as np
import tensorflow as tf
def write_to_file(filename, data):
    """Write TensorFlow tensor data to a text file without truncation."""
    tensor_data = data.numpy() if isinstance(data, tf.Tensor) else data

    # Open the file and write the tensor data
    with open(filename, 'w') as f:
        if isinstance(data, np.ndarray):
            np.savetxt(f, data, fmt="%.6f")
        elif isinstance(data, list):
            for item in data:
                f.write(f"{item}\n")
        else:
            f.write(str(data))

In [10]:
import time
import os
from base import fast_QPT_recreate
from base import optimize_algorithm
experiment_folder = 'results/experiment_new/comparison'

for n in range (3, 4):
    if (experiment_folder == ''):
        break
    else:
        write_folder = os.path.join(experiment_folder, str(n) + "_qubits")
        if not os.path.exists(write_folder):
            os.makedirs(write_folder)
    print(f"N={n}")

    rho_list, measurement_operators, epsilon, kraus_operators = initExperiment(n)
    
    # ----------------------------
    # Fast QPT based on multi-shot measurement
    start_time = time.time()
    kraus_operators_res, cost_dict = fast_QPT_recreate.optimize_adam_kraus(measurement_operators, rho_list, epsilon, kraus_operators, n, 0.06, num_loop=200)
    end_time = time.time()
    multi_shot_time = end_time - start_time
    
    rho3_list = epsilon_rho.calculate_set_from_kraus_operators(kraus_operators_res, rho_list, epsilon)
    rho2_list, rho2_kraus_list = calculate_rho2_lists(rho_list, epsilon, kraus_operators_res)
    
    mean_fidelity_rho_rho3 = metrics.mean_fidelity(rho3_list, rho_list)
    mean_fidelity_rho2_rho2 = metrics.mean_fidelity(rho2_kraus_list, rho2_list)

    # ----------------------------
    # Fast QPT based on single-shot measurement
    start_time = time.time()
    kraus_operators_res_our_method, cost_dict_our_method = optimize_algorithm.optimize_adam_kraus_set(rho_list, epsilon, kraus_operators, n, alpha=0.08, num_loop=200)
    end_time = time.time()
    single_shot_time = end_time - start_time

    rho3_list = epsilon_rho.calculate_set_from_kraus_operators(kraus_operators_res_our_method, rho_list, epsilon)
    rho2_list, rho2_kraus_list = calculate_rho2_lists(rho_list, epsilon, kraus_operators_res_our_method)
    
    mean_fidelity_rho_rho3_our_method = metrics.mean_fidelity(rho3_list, rho_list)
    mean_fidelity_rho2_rho2_our_method = metrics.mean_fidelity(rho2_kraus_list, rho2_list)

    write_to_file(os.path.join(write_folder, "rho_list.txt"), rho_list)
    write_to_file(os.path.join(write_folder,"epsilon.txt"), epsilon)
    write_to_file(os.path.join(write_folder,"measurement_operators.txt"), measurement_operators)
    write_to_file(os.path.join(write_folder,"init_kraus_operators.txt"), kraus_operators)

    write_to_file(os.path.join(write_folder,"kraus_operators.txt"), kraus_operators_res)
    write_to_file(os.path.join(write_folder,"cost_dict.txt"), cost_dict)
    write_to_file(os.path.join(write_folder,"time.txt"), multi_shot_time)

    write_to_file(os.path.join(write_folder,"kraus_operators_ours.txt"), kraus_operators_res_our_method)
    write_to_file(os.path.join(write_folder,"cost_dict_ours.txt"), cost_dict_our_method)
    write_to_file(os.path.join(write_folder,"time_ours.txt"), single_shot_time)

    write_to_file(os.path.join(write_folder,"mean_fidelity_rho_rho3.txt"), mean_fidelity_rho_rho3.numpy())
    write_to_file(os.path.join(write_folder,"mean_fidelity_rho2_rho2.txt"), mean_fidelity_rho2_rho2.numpy())
    write_to_file(os.path.join(write_folder,"mean_fidelity_rho_rho3_ours.txt"), mean_fidelity_rho_rho3_our_method.numpy())
    write_to_file(os.path.join(write_folder,"mean_fidelity_rho2_rho2_ours.txt"), mean_fidelity_rho2_rho2_our_method.numpy())

N=3
Generated 216 of (8, 8) rho.
Generated 216 of (8, 8) M.
Generated (8, 8) epsilon.
Generated 8 of (8, 8) kraus operators.
tf.Tensor(0.863179446656344, shape=(), dtype=float64)


KeyboardInterrupt: 

In [11]:
from base import epsilon_rho
from base import metrics
rho_test = generator_haar.generate_rho_haar(n)

rho2 = epsilon_rho.calculate_from_unitary(rho_test, epsilon)

rho2_ours = epsilon_rho.calculate_from_kraus_operators(rho_test, kraus_operators_res_our_method)

rho2_theirs = epsilon_rho.calculate_from_kraus_operators(rho_test, kraus_operators_res)

#print (rho2)

print ("Their method: \n")
#print (rho2_theirs)
print(metrics.compilation_trace_fidelity(rho2, rho2_theirs))

print ("Our method: \n")
#print (rho2_ours)
print(metrics.compilation_trace_fidelity(rho2, rho2_ours))

2025-03-01 18:49:26.972043: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Matrix size-incompatible: In[0]: [4,4], In[1]: [8,8]


InvalidArgumentError: {{function_node __wrapped__MatMul_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [4,4], In[1]: [8,8] [Op:MatMul] name: 

In [ ]:
def check_rho_properties(rho_list):
    for i, rho in enumerate(rho_list):
        is_hermitian = np.allclose(rho, np.transpose(np.conjugate(rho)))  # Check if ρ = ρ†
        eigenvalues = np.linalg.eigvalsh(rho)  # Compute eigenvalues
        is_positive_semidefinite = np.all(eigenvalues >= -1e-10)  # Allow small numerical errors
        trace_one = np.isclose(np.trace(rho), 1)  # Check if Tr(ρ) ≈ 1
        
        print(f"ρ[{i}]: Hermitian = {is_hermitian}, Positive Semi-definite = {is_positive_semidefinite}, Trace = {np.trace(rho)}")
        print(f"Eigenvalues: {eigenvalues}\n")

check_rho_properties(rho_list)


In [ ]:
import tensorflow as tf
import numpy as np

def check_kraus_validity(kraus_operators):
    """Check if sum(K_i^† K_i) = I"""
    dim = kraus_operators[0].shape[0]  # Kích thước của ma trận
    identity = tf.eye(dim, dtype=tf.complex128)  # Ma trận đơn vị
    summation = sum(tf.linalg.adjoint(K) @ K for K in kraus_operators)  # ∑ K_i^† K_i

    # So sánh với ma trận đơn vị
    error = tf.linalg.norm(summation - identity).numpy().real
    
    if np.isclose(error, 0, atol=1e-6):
        return
    else:
        print("❌ Invalid Kraus operators! The sum does not equal identity.")

# Kiểm tra
check_kraus_validity(kraus_operators)


In [ ]:
import numpy as np

def check_valid_measurements(M_list, d):
    """
    Kiểm tra xem danh sách các measurement operators M_list có hợp lệ không.
    
    Điều kiện hợp lệ:
    1. M_i phải Hermitian
    2. M_i phải Positive Semi-Definite (PSD)
    3. Tổng M_i phải bằng ma trận đơn vị I (trong trường hợp POVM)
    
    Args:
        M_list: Danh sách các phép đo
        d: Kích thước không gian Hilbert (ví dụ: d=2^n cho hệ n qubit)
        
    Returns:
        None (In kết quả)
    """
    I = np.eye(d)  # Ma trận đơn vị cùng kích thước

    completeness_check = np.zeros((d, d), dtype=complex)  # Tổng của các M_i

    for i, M in enumerate(M_list):
        is_hermitian = np.allclose(M, M.conj().T)  # Kiểm tra Hermitian
        eigvals = np.linalg.eigvalsh(M)  # Lấy eigenvalues
        is_psd = np.all(eigvals >= -1e-10)  # Kiểm tra PSD (chấp nhận sai số nhỏ)

        completeness_check += M  # Cộng vào tổng

        print(f"M[{i}]: Hermitian = {is_hermitian}, Positive Semi-Definite = {is_psd}")
        print(f"  Eigenvalues: {eigvals}\n")

    # Kiểm tra tổng M_i có bằng I không (chỉ cần thiết nếu là POVM)
    is_complete = np.allclose(completeness_check, I)
    print(completeness_check)
    print(f"Completeness Check (ΣM_i = I): {is_complete}")
check_valid_measurements(M_list, 2**n)
